In [22]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2, os, math, time
from datetime import timedelta
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize

### Configuration

In [2]:
###Configuration
"""
Data Configurations/Paths
"""
img_dir_patch="./SD/predicted_patches"
img_dir_orig = "./SD/original_images"
img_dir="../../original_images/SD"
model50_SD = 'SD/50kSD_Model.ckpt'
model50_local_top_1 = 'SD/50k_local_top_Model.ckpt'
model50_left_mask = 'SD/50k_left_mask.ckpt'

img_type = "original"
# img_type = "patch"

##
# Convolutional Layer 1.
filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 2          # Convolution filters are 2 x 2 pixels.
num_filters2 = 32         # There are 32 of these filters.

# Convolutional Layer 3.
filter_size3 = 2          # Convolution filters are 2 x 2 pixels.
num_filters3 = 64         # There are 64 of these filters.

# Convolutional Layer 4.
filter_size4 = 2          # Convolution filters are 2 x 2 pixels.
num_filters4 = 128         # There are 128 of these filters.

# Fully-connected layer.
fc_size = 2000             # Number of neurons in fully-connected layer.

# We know that images are 60 pixels in each dimension.
# img_size = 8 * 4

# Images are stored in one-dimensional arrays of this length.
img_size_flat = 10 * 10

# Number of colour channels for the images: 3 channel for RGB.
num_channels = 3

# Tuple with height and width of images used to reshape arrays.
img_shape = (10, 10, num_channels)

# Number of classes, one class for same or different image
num_classes = 10*10
orig_patch_size = (2, 2, 3)
npatches = 1

### Load Data

In [36]:
def change_label_dimensions(labels):
    label_temp = np.zeros((len(labels), 2))
    
    for idx in range(0, len(labels)):
        if labels[idx] == 1:
            label_temp[idx][1] = 1
        else:
            label_temp[idx][0] = 1
    
    return label_temp

def load_data(img_dir):
        list_of_orig_imgs = []
        list_of_pred_imgs = []
        list_of_labels = []
        list_same_diff = []
        list_img_keys = []
        for img in os.listdir(img_dir):
            
            img_path = os.path.join(img_dir, img)
            list_same_diff.append(int(os.listdir(img_path)[0]))
            list_img_keys.append(img)
            img_path = img_path + "/" + os.listdir(img_path)[0]
            for img_label in os.listdir(img_path):
                img_data = os.path.join(img_path, img_label)
                if img_label == "img":
#                     print(img_data + "/img.png")
                    list_of_orig_imgs.append(img_data + "/img.png")
                    list_of_pred_imgs.append(img_data + "/predicted_mask.png")
                else:
                    list_of_labels.append([os.path.join(img_data, label) for label in os.listdir(img_data)])

        data_imgs = np.array([list_of_orig_imgs, list_of_pred_imgs])
        data_labels = np.array(list_of_labels)
        data_same_diff = np.array(list_same_diff)
        data_img_keys = np.array(list_img_keys)

        return data_imgs, data_labels, data_same_diff, data_img_keys

    
def get_batch_images(data_orig, data_pred, label, same_diff, img_keys, rshp, grey_scale):
        list_of_orig_imgs = []
        list_of_pred_imgs = []
        list_of_labels = []
        list_of_same_diff = []
        list_of_img_keys = []
        for img_orig, img_pred, lbl, img_type, img_key in zip(data_orig, data_pred, label, same_diff, img_keys):
            if (grey_scale):
                orig_img = cv2.imread(img_orig)
                predicted_msk = cv2.imread(img_pred, cv2.IMREAD_GRAYSCALE)
            else:
                orig_img = cv2.imread(img[0])
            
#             print(lbl, type(lbl))
            orig_lbl = cv2.imread(lbl, cv2.IMREAD_GRAYSCALE)
            if orig_img is None or orig_lbl is None:
                    print ("Unable to read image{} or {}".format(img[0], lbl))
                    continue
            
#             if (grey_scale):
#                 orig_lbl = rgb2grey(orig_lbl)

            flattened_orig_img = orig_img.flatten()
            flattened_pred_img = predicted_msk.flatten()
            flattened_lbl = orig_lbl.flatten()
            
#             if grey_scale:
#                 flattened_lbl = np.reshape(flattened_lbl, [10, 10])
#                 print(flattened_lbl)
#                 flattened_lbl = normalize(flattened_lbl)
#                 print(flattened_lbl)
            
            list_of_orig_imgs.append(np.asarray(flattened_orig_img, dtype=np.float32))
            list_of_pred_imgs.append(np.asarray(flattened_pred_img, dtype=np.float32))
        
            list_of_labels.append(np.asarray(flattened_lbl, dtype=np.float32))
            list_of_same_diff.append(img_type)
            list_of_img_keys.append(img_key)

        data_labels = np.array(list_of_labels)
        data_imgs = np.array([list_of_orig_imgs, list_of_pred_imgs])
        data_img_type = np.array(list_of_same_diff)
        data_img_keys = np.array(list_of_img_keys)
        
        """this function call locates top left location of each patch in the mask and return. Comment it if contents are required."""
#         print(data_labels.shape)
#         data_labels = get_patch_loc(data_labels)
        
        return data_imgs, data_labels, data_img_type, data_img_keys



### Next Batch Own Implementation

In [4]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data[0]))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_orig = data[0, :]
    data_pred = data[1, :]
    data_orig_shuffle = [data[0, i] for i in idx]
    data_pred_shuffle = [data[1, i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_orig_shuffle), np.asarray(data_pred_shuffle), np.asarray(labels_shuffle)

### Helper Functions for Network

In [23]:
def new_weights(shape, layer_name):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.Variable(initializer(shape), name=layer_name+'_W')

def new_bias(length, layer_name):
    return tf.Variable(tf.constant(0.05, shape=[length]), name=layer_name+'_b')

def new_conv_layer(input,
                   num_input_channels,
                   filter_size,
                   num_filters,
                   name_scope,
                   layer_name='',
                   use_pooling=True):

    with tf.name_scope(name_scope):
        shape = [filter_size, filter_size, num_input_channels, num_filters]
        weights = new_weights(shape, layer_name)
        biases = new_bias(num_filters, layer_name)

        layer = tf.add(tf.nn.conv2d(input=input, filter=weights, strides=[1,1,1,1], padding='SAME'), biases, name=layer_name)

        if use_pooling:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 3, 3, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME', name=layer_name+'_max')
        layer = tf.nn.relu(layer, name=layer_name+'_activation')

    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features

def new_fc_layer(input,
                num_inputs,
                num_outputs,
                name_scope,
                layer_name='',
                use_relu=True):
    
    with tf.name_scope(name_scope):
        weights = new_weights([num_inputs, num_outputs], layer_name)
        biases = new_bias(num_outputs, layer_name)

        layer = tf.add(tf.matmul(input, weights),biases,name=layer_name)
    #     layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer, layer_name+'_activation')
    
    return layer

def normalise(tensor):
    return tf.div(
   tf.subtract(
      tensor, 
      tf.reduce_min(tensor)
   ), 
   tf.subtract(
      tf.reduce_max(tensor), 
      tf.reduce_min(tensor)
   )
)

def normalized(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

In [6]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat*num_channels], name='x')
x_image = tf.reshape(x, [-1, 10, 10, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

### Convolution Layers

In [7]:
layer1_conv1, weights_conv1 = new_conv_layer(input=x_image,
                                            num_input_channels=num_channels,
                                            filter_size=filter_size1,
                                            num_filters=num_filters1,
                                             name_scope = 'conv',
                                             layer_name = 'conv1',
                                            use_pooling=True)

layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
                                           num_input_channels=num_filters1,
                                           filter_size=filter_size2,
                                           num_filters=num_filters2,
                                             name_scope = 'conv',
                                             layer_name = 'conv2',
                                              
                                           use_pooling=True)

layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
                                           num_input_channels=num_filters2,
                                           filter_size=filter_size3,
                                           num_filters=num_filters3,
                                             name_scope = 'conv',
                                             layer_name = 'conv3',
                                              
                                           use_pooling=True)

layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
                                           num_input_channels=num_filters3,
                                           filter_size=filter_size4,
                                           num_filters=num_filters4,
                                             name_scope = 'conv',
                                             layer_name = 'conv4',
                                           use_pooling=True)

In [8]:
layer_flat, num_features = flatten_layer(layer4_conv4)

### Fully Connected Layers

In [9]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         name_scope = 'fc',
                         layer_name = 'fc1',
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=fc_size,
                         name_scope = 'fc',
                         layer_name = 'fc2',
                         use_relu=False)

layer_fc3 = new_fc_layer(input=layer_fc2,
                         num_inputs=fc_size,
                         num_outputs=fc_size,
                         name_scope = 'fc',
                         layer_name = 'fc3',
                         use_relu=False)

layer_fc4 = new_fc_layer(input=layer_fc3,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         name_scope = 'fc',
                         layer_name = 'fc4',
                         use_relu=False)

# drop_out = tf.nn.dropout(layer_fc4, 0.5, name="drop_out")

y_pred = tf.nn.softmax(layer_fc4, name="softmax_output")

### Cost Function

In [10]:
cost = tf.reduce_mean(tf.square(y_true - y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# ## some more performance measures
correct_prediction = tf.equal(y_pred, y_true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Run Tensorflow on Defined Graph

In [37]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)
session = tf.Session()
init = tf.global_variables_initializer()
session.run(init)
train_labels = train_labels[:][:, 3]
train_orig_data = train_data[0, :]
train_pred_data = train_data[1, :]
img_type = img_type[:]
img_keys = img_keys[:]
total_imgs = len(img_type)
train_batch_size = 64

def optimize(num_epochs, save_model=True,save_name= "base_model",restore_model=False,restore_name=None):
    total_iterations = 0
    done_train_imgs = 0
    start_time = time.time()
    start_ = 0
    end_ = train_batch_size    
    plot_accuracy=[]
    plot_accuracy_epoch=[]
    plot_training_size=[]
    plot_training_size_epoch=[]
    saver = tf.train.Saver()
    sum_accuracy = 0.0
    n = 1
    
        #to save the model
    for i in range(0, num_epochs):   
        start_batch=0
        end_batch = train_batch_size
        
        print("Epoch:", i + 1)
        
        if restore_model==True:
            if restore_name==None:
                print("No model file specified")
                return
            else:
                saver.restore(session,restore_name)
        
        sum_accuracy = 0.0
        n = 1
        while end_batch < total_imgs:
            train_orig = train_orig_data[start_batch:end_batch]
            train_pred = train_pred_data[start_batch:end_batch]
            labels = train_labels[start_batch:end_batch]
#             print('Labels:', labels)
            img_type_lbl = img_type[start_:end_]
            img_key = img_keys[start_:end_]
            dims = (len(train_orig), num_classes, num_channels)
            train, labels, img_type_lbl, img_key = get_batch_images(train_orig, train_pred, labels, img_type_lbl, img_key, dims, True)
            if not len(train) and not len(labels):
                print("All images have been processed.")
                break;

            x_orig_batch, x_pred_batch, y_true_batch = next_batch(len(train[0]), train, labels)
            feed_dict_train = {x: x_orig_batch,
                       y_true: y_true_batch}
            
            session.run(optimizer, feed_dict=feed_dict_train)
    
            acc,co = session.run([accuracy, cost], feed_dict=feed_dict_train)
            sum_accuracy += acc
            n+=1
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}, Loss: {2:>.4f}"
            print(msg.format(end_batch + 1, acc, co))
            if i == num_epochs - 1:
                plot_accuracy.append(acc)
                plot_training_size.append(end_batch + 1)

            start_batch += train_batch_size
            end_batch += train_batch_size
    
        if save_model==True:
            if save_name==None:
                print("No model specified, model not being saved")
                return
            else:
                save_path = saver.save(session, save_name)
                restore_model = True
                print("Model saved in file: %s" % save_name)
        plot_accuracy_epoch.append(sum_accuracy/n)
        plot_training_size_epoch.append(i + 1)
    
    end_time = time.time()
    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))  
    print(plot_accuracy)
    print(plot_training_size)
    print(plot_accuracy_epoch)
    print(plot_training_size_epoch)

### Performance/Optimization

In [ ]:
save_model = True
save_name = model50_left_mask
restore_model=True
restore_name=model50_left_mask
# init = tf.global_variables_initializer()
# session.run(init)

optimize(20, save_model=True,save_name=model50_left_mask,restore_model=restore_model,restore_name=model50_left_mask)

Epoch: 1
INFO:tensorflow:Restoring parameters from SD/50k_left_mask.ckpt
Optimization Iteration:     65, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:    129, Training Accuracy:   0.0%, Loss: 2711.8318
Optimization Iteration:    193, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:    257, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:    321, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:    385, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:    449, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:    513, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:    577, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:    641, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:    705, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:    769, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:    

Optimization Iteration:   7041, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:   7105, Training Accuracy:   0.0%, Loss: 2711.7742
Optimization Iteration:   7169, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:   7233, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:   7297, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:   7361, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:   7425, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:   7489, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:   7553, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:   7617, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:   7681, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:   7745, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:   7809, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  

Optimization Iteration:  14081, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14145, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  14209, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14273, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  14337, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  14401, Training Accuracy:   0.0%, Loss: 2712.3523
Optimization Iteration:  14465, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  14529, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  14593, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14657, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  14721, Training Accuracy:   0.0%, Loss: 2712.4099
Optimization Iteration:  14785, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  14849, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  

Optimization Iteration:  21121, Training Accuracy:   0.0%, Loss: 2711.8318
Optimization Iteration:  21185, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21249, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21313, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21377, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  21441, Training Accuracy:   0.0%, Loss: 2712.3523
Optimization Iteration:  21505, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  21569, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  21633, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21697, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  21761, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  21825, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  21889, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  

Optimization Iteration:  28161, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  28225, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  28289, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  28353, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  28417, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  28481, Training Accuracy:   0.0%, Loss: 2711.8896
Optimization Iteration:  28545, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:  28609, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  28673, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  28737, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  28801, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  28865, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  28929, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  

...

Optimization Iteration:  42625, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  42689, Training Accuracy:   0.0%, Loss: 2711.8318
Optimization Iteration:  42753, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:  42817, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  42881, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  42945, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  43009, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  43073, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  43137, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:  43201, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  43265, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  43329, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  43393, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  

Optimization Iteration:  49665, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  49729, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  49793, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  49857, Training Accuracy:   0.0%, Loss: 2711.7742
Optimization Iteration:  49921, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  49985, Training Accuracy:   0.0%, Loss: 2712.0054
Model saved in file: SD/50k_left_mask.ckpt
Epoch: 10
INFO:tensorflow:Restoring parameters from SD/50k_left_mask.ckpt
Optimization Iteration:     65, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:    129, Training Accuracy:   0.0%, Loss: 2711.8318
Optimization Iteration:    193, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:    257, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:    321, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:    385, Training Accuracy:   0.0%,

Optimization Iteration:   6593, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:   6657, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:   6721, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:   6785, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:   6849, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:   6913, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:   6977, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:   7041, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:   7105, Training Accuracy:   0.0%, Loss: 2711.7742
Optimization Iteration:   7169, Training Accuracy:   0.0%, Loss: 2711.9475
Optimization Iteration:   7233, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:   7297, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:   7361, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  

Optimization Iteration:  13633, Training Accuracy:   0.0%, Loss: 2712.3523
Optimization Iteration:  13697, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  13761, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  13825, Training Accuracy:   0.0%, Loss: 2711.8896
Optimization Iteration:  13889, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  13953, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  14017, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14081, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14145, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  14209, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  14273, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  14337, Training Accuracy:   0.0%, Loss: 2712.1787
Optimization Iteration:  14401, Training Accuracy:   0.0%, Loss: 2712.3523
Optimization Iteration:  

Optimization Iteration:  20673, Training Accuracy:   0.0%, Loss: 2712.1208
Optimization Iteration:  20737, Training Accuracy:   0.0%, Loss: 2712.2366
Optimization Iteration:  20801, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  20865, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  20929, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  20993, Training Accuracy:   0.0%, Loss: 2712.0632
Optimization Iteration:  21057, Training Accuracy:   0.0%, Loss: 2711.7742
Optimization Iteration:  21121, Training Accuracy:   0.0%, Loss: 2711.8318
Optimization Iteration:  21185, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21249, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21313, Training Accuracy:   0.0%, Loss: 2712.0054
Optimization Iteration:  21377, Training Accuracy:   0.0%, Loss: 2712.2944
Optimization Iteration:  21441, Training Accuracy:   0.0%, Loss: 2712.3523
Optimization Iteration:  

In [13]:
def restore_see_layer(orig, model_name=None, var_name=None):
    with tf.Session('', tf.Graph()) as s:
        with s.graph.as_default():
            if ((model_name != None) and var_name != None):
                saver = tf.train.import_meta_graph(model_name+".meta")
                saver.restore(s, model_name)
#                 print(pred.shape)
                fd = {'x:0': orig}
#                 print(fd.shape)
                var_name=var_name+":0"
                
                result = 0
                result = s.run(var_name, feed_dict=fd)
    return result

def see_output_grey(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:]
    fig = plt.figure(figsize=figsize)
    plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()


def see_output(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:,:]
    fig = plt.figure(figsize=figsize)
    if cmap == "gray":
        plt.imshow(img_x, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()
    
def save_patch_images(img_x1, lbl_x1, index):
    if not os.path.exists('./SD/predicted_patches/' + str(index)):
        os.makedirs('./SD/predicted_patches/' + str(index))
        os.makedirs('./SD/predicted_patches/' + str(index) + "/" + str(lbl_x1))
        
    plt.imsave('./SD/predicted_patches/' + str(index) + "/" + str(lbl_x1) + '/img.png', np.squeeze(img_x1))
    

def predict_nd_save(train, labels, img_type_lbl, img_key, start_idx):

    for index in range(0, len(train)):
        img_x = train[index:index+1, :]
        lbl_x = labels[index:index+1, :]
        img_type_x = img_type_lbl[index]
        img_key_x = img_key[index]
        prediction = restore_see_layer(ix=img_x,model_name=model2_50000,var_name='Softmax')
        prediction = np.reshape(prediction, (1, 4, 2, 1))   
        save_patch_images(prediction, img_type_x, img_key_x)

In [ ]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)
train_mask_labels = train_labels[:][:, 1]
train_patch_labels = train_labels[:][:, 0]

batch_s = 64
total_iterations = 0
start_ = 0
end_ = batch_s
np.set_printoptions(suppress=True)

if not os.path.exists('./SD/predicted_patches'):
    os.makedirs('./SD/predicted_patches')

while True:
    train = train_data[start_:end_]
    labels = train_patch_labels[start_:end_]
    img_type_lbl = img_type[start_:end_]
    img_key = img_keys[start_:end_]
    dims = (batch_s, num_classes, num_channels)
    train, labels, img_type_lbl, img_key = get_batch_images(train, labels, img_type_lbl, img_key, dims, True)
    predict_nd_save(train, labels, img_type_lbl, img_key, start_)
    
    #do my stuff
    if len(train_data) < start_ + batch_s:
        print("{} Images have been processed.".format(total_iterations))
        break
    
    total_iterations +=batch_s
    start_ = end_
    end_ = end_ + batch_s

In [14]:
def get_tensors(graph=tf.get_default_graph()):
    return [t for op in graph.get_operations() for t in op.values()]
get_tensors()

[<tf.Tensor 'x:0' shape=(?, 300) dtype=float32>,
 <tf.Tensor 'Reshape/shape:0' shape=(4,) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(?, 10, 10, 3) dtype=float32>,
 <tf.Tensor 'y_true:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'ArgMax/dimension:0' shape=() dtype=int32>,
 <tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>,
 <tf.Tensor 'conv/random_uniform/shape:0' shape=(4,) dtype=int32>,
 <tf.Tensor 'conv/random_uniform/min:0' shape=() dtype=float32>,
 <tf.Tensor 'conv/random_uniform/max:0' shape=() dtype=float32>,
 <tf.Tensor 'conv/random_uniform/RandomUniform:0' shape=(4, 4, 3, 16) dtype=float32>,
 <tf.Tensor 'conv/random_uniform/sub:0' shape=() dtype=float32>,
 <tf.Tensor 'conv/random_uniform/mul:0' shape=(4, 4, 3, 16) dtype=float32>,
 <tf.Tensor 'conv/random_uniform:0' shape=(4, 4, 3, 16) dtype=float32>,
 <tf.Tensor 'conv/conv1_W:0' shape=(4, 4, 3, 16) dtype=float32_ref>,
 <tf.Tensor 'conv/conv1_W/Assign:0' shape=(4, 4, 3, 16) dtype=float32_ref>,
 <tf.Tensor 'conv/conv1_W/read:0' sh

In [15]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)

In [25]:
train_orig = train_data[0, 0:1]
train_pred = train_data[1, 0:1]
labels = train_labels[0:1, 3]
print(labels)
img_type_lbl = img_type[0:1]
img_key = img_keys[0:1]
dims = (1, num_classes, num_channels)
train, labels, img_type_lbl, img_key = get_batch_images(train_orig, train_pred, labels, img_type_lbl, img_key, dims, True)
# train[0:1]
# img_x = cv2.imread("../../original_images/SD/15970/0/img/predicted_mask.png")

# img_x = np.expand_dims(img_x[:,:,0].flatten(), axis=0)
np.set_printoptions(suppress=True)
output_cl1 = restore_see_layer(orig=np.expand_dims(train[0][0], axis=0), model_name=model50_left_mask,var_name='softmax_output')
last_activs = restore_see_layer(orig=np.expand_dims(train[0][0], axis=0), model_name=model50_left_mask,var_name='fc_3/fc4')

norm_labels = normalized(labels)
norm_last_activs = normalized(last_activs)
print(norm_labels)
print(norm_last_activs)


# see_output_grey(np.reshape(labels, [1, 10,10]))
# see_output_grey(np.reshape(last_activs, [1, 10,10]))

['../../original_images/SD/15970/0/labels/mask_patch_1.png']
INFO:tensorflow:Restoring parameters from SD/50k_left_mask.ckpt
INFO:tensorflow:Restoring parameters from SD/50k_left_mask.ckpt
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
[[0.38653535 0.4057966  0.36508724 0.39345306 0.37602678 0.39907455
  0.38074392 0.3888056  0.34278107 0.3691759  0.35886088 0.35111886
  0.4131743  0.38736975 0.3912501  0.37365112 0.39929292 0.38387197
  0.33868736 0.36247176 0.36073625 0.41177985 0.40874842 0.38679218
  0.4088831  0.41795057 0.3343268  0.39654467 0.35719928 0.38529855
  0.35577607 0.4045641  0.41749442 0.39907858 0.40314427 0.37935904
  0.35090786 0.37142724 0.34904832 0.3900527  0.4318654  0.4033997
  1.         0.38182607 